In [1]:
import numpy as np
import matplotlib 
matplotlib.use('Qt5Agg') 
import matplotlib.pyplot as plt
import random
import re

from os import listdir
from os.path import isfile, join
from itertools import filterfalse

In [2]:
def read_qps(file_path: str, file_name: str):
    def is_data_row(line):
        return bool(re.match(r"^\s*\d*\.?\d+e?[-+]?\d*\s+\d*\.?\d+e?[-+]?\d*\s*$", line))
    pressure = []
    volume = []    
    with open(file_path, 'r') as file:
        for line in file:
            if is_data_row(line):
                p, v = map(float, line.split())
                pressure.append(p)
                volume.append(v)
    
    pressure = np.array(pressure)
    volume = np.array(volume)
    return [pressure, volume, file_name]

def separate_branches(iso):
    pressure = iso[0]
    volume = iso[1]
    max_p_idx = np.where(pressure == max(pressure))[0][0]
    isotherm = {"adsorption_p": pressure[:max_p_idx+1], "adsorption": volume[:max_p_idx+1],
                "desorption_p": pressure[max_p_idx:][::-1], "desorption": volume[max_p_idx:][::-1], "file_name": iso[2]}
    return isotherm
    
data_path = "../data/experimental_isotherms/qps"
file_names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
data = [read_qps(join(data_path, f), f) for f in file_names]
isotherms = [separate_branches(d) for d in data if len(d[0]) != 0]
print(len(isotherms))

1143


In [3]:
def filter_number_of_points(isotherm, min_number_of_points=15):
    if len(isotherm["adsorption_p"]) <= min_number_of_points:
        return False
    return True

def filter_pressure_fall(isotherm):
    for i in range(len(isotherm["adsorption_p"])-1):
        if isotherm["adsorption_p"][i] >= isotherm["adsorption_p"][i+1]:
            return False
    return True

def filter_adsorption_fall(isotherm):
    for i in range(len(isotherm["adsorption_p"])-1):
        if isotherm["adsorption"][i+1]/isotherm["adsorption"][i] < 1:
            return False
    return True

In [4]:
filters = [filter_number_of_points, filter_pressure_fall, filter_adsorption_fall]
clean_data = isotherms.copy()
for f in filters:
    print(f"{f.__name__} : {len(list(filterfalse(f, clean_data)))}")
    clean_data = list(filter(f, clean_data))
print(f"{len(isotherms)} ==FILTERS==> {len(clean_data)}")

filter_number_of_points : 53
filter_pressure_fall : 188
filter_adsorption_fall : 101
1143 ==FILTERS==> 801


C:\Users\ivano\AppData\Local\Temp\ipykernel_11088\3598729498.py:14: RuntimeWarning: invalid value encountered in scalar divide
  if isotherm["adsorption"][i+1]/isotherm["adsorption"][i] < 1:
C:\Users\ivano\AppData\Local\Temp\ipykernel_11088\3598729498.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  if isotherm["adsorption"][i+1]/isotherm["adsorption"][i] < 1:


In [5]:
i = random.randint(0, len(clean_data)-1)
plt.plot(clean_data[i]["adsorption_p"], clean_data[i]["adsorption"], marker=".")
plt.plot(clean_data[i]["desorption_p"], clean_data[i]["desorption"], marker=".")
plt.show()